In [1]:
from tinydb import TinyDB, Query
import os
import json
import numpy as np

class DB:
    def __init__(self, db_name: str) -> None:
        self.db_instance = TinyDB(db_name)
        self.tables = {field: self.db_instance.table(field) for field in ["reality", "objects", "rules"]}
        
        # Defaults
        self.OBJECTS = {
        "name": None,
        "id": None,
        "cat": "object",
        "desc": "An object.",
        "initRequire": [],
        "constRequire": [],
        "initGains": [],
        "constGains": [],
        "replace": [],
        "restrict": [],
        "visible": True
        }
        self.RULES = {
            "dismantleGain": 0.5
        }
        self.REALITY = {
            "id": None,
            "type": None,
            "amount": None
        }

class Compiler(DB):
    def __init__(self, db_name: str, scenario_dir: str) -> None:
        super().__init__(db_name)
        self.scenario_dir = scenario_dir

    def fill_empty(self, obj: dict, default_vals: dict) -> dict:
        for key in default_vals.keys():
            if key not in obj.keys():
                obj.update({key: default_vals[key]})
        return obj


    def integrate_objects(self, content: list[dict]) -> list[dict]:
        error_handle = []
        for element in content: 
            if len([el for el in element.keys() if el in ["name", "id"]]) == 2:
                content[content.index(element)] = self.fill_empty(element, self.OBJECTS)
            else:
                error_handle.append(json.dumps(element))

        if not error_handle:
            return content
        else:
            err = '\n'.join(error_handle)
            print(f"No mandatory fields id and name in objects: indexes: {err}")
                    

    def compile_scenario(self) -> None:
        valid_scenario_files = [f for f in os.listdir(self.scenario_dir) if f.endswith(".json")]
        
        for file in valid_scenario_files:
            with open(f"{self.scenario_dir}/{file}", "r") as f:
                content = json.loads(f.read())
                for key in content.keys():
                    key = key.lower()
                    
                    if key == "objects":
                        integrated_content = self.integrate_objects(content[key])
                        self.tables[key].insert_multiple(integrated_content)
                    elif key == "rules":
                        self.tables[key].insert(self.fill_empty(content[key], self.RULES))
                    elif key == "reality":
                        for real in content[key]:
                            if not np.isin(list(real.keys()), list(self.REALITY.keys())).all():
                                print("Some reality elements does not fit schema.")
                                exit()
                        self.tables[key].insert_multiple(content[key])


In [33]:
class Queue(DB):
    def __init__(self, db_name: str, queue: list[dict]) -> None:
        super().__init__(db_name)
        self.queue = queue
        self.agg = self.queue_agg()

    def queue_agg(self) -> dict:
        balance_mode = ["initRequire", "initGains"]
        obj_mode = [m["targetType"] for m in self.queue]
        bridge = {m: [q for q in self.queue if q["targetType"] == m] for m in obj_mode}

        summary = {"intoReality": [], "intoVoid": [], "initRequire": [], "initGains": []} 

        for mode in bridge:
            id_list = [i["targetId"] for i in bridge[mode]]
            search = self.tables[mode].search(Query()["id"].one_of(id_list))

            if "intoVoid" in [x["action"] for x in bridge[mode]]:
                dismantle_rate = self.tables["rules"].all()[0]["dismantleGain"]


            for item in bridge[mode]:
                buckets_id = [i["id"] for i in summary[item["action"]]]
                summary[item["action"]].append({"id": item["targetId"], "type": item["targetType"]})
                
                balance = {m: [s[m] for s in search if s["id"] == item["targetId"]][0] for m in balance_mode}
                
                if item["action"] == "intoVoid":
                    for void in balance["initRequire"]:
                        summary["initGains"].append({
                            "id": void["id"], 
                            "type": "objects", 
                            "amount": void["amount"] * dismantle_rate})
                else:
                    [[summary[m].append(i) for i in balance[m]] for m in balance_mode]


        for mode in summary.keys():
            new_list = []

            for item in summary[mode]:
                current_id = [i["id"] for i in new_list]
                if mode not in balance_mode:
                    item["amount"] = 1
                    
                if not new_list:
                    new_list.append(item)
                elif item["id"] in current_id:
                    target_index = current_id.index(item["id"])
                    new_list[target_index]["amount"] += item["amount"]
                else:
                    new_list.append(item)
            
            summary[mode] = new_list
        
        return summary


    def check(self):
        reality = self.tables["reality"].all()
        reality_types = [t["type"] for t in reality]
        err = []
        
        for void in self.agg["intoVoid"]:
            if not [r for r in reality if r["id"] == void["id"] and r["type"] == void["type"]]:
                msg = f"""{void["id"]} of type {void["type"]} not in reality."""
                err.append(msg)

        for cost in self.agg["initRequire"]:
            req_in_real = [r for r in reality if r["id"] == cost["id"] and r["type"] == cost["type"]][0]
            gain = [g for g in self.agg["initGains"] if g["id"] == cost["id"] and g["type"] == cost["type"]]
            gain = {"amount": 0} if not gain else gain[0]
            
            if cost["amount"] > req_in_real["amount"] + gain["amount"]:
                msg = f"""Need {cost["amount"]} {cost["id"]} of type {cost["type"]}. Have {req_in_real["amount"] + gain["amount"]}"""
                err.append(msg)
                

        return err

In [34]:
q = [
    {"action": "intoReality", "targetId": "hut_wood", "targetType": "objects"},
    {"action": "intoReality", "targetId": "hut_wood", "targetType": "objects"}
]


que = Queue("db.json", q)

In [35]:
que.check()

['Need 20 resource_wood of type objects. Have 5',
 'Need 10 resource_workforce of type objects. Have 5']

In [7]:
que.check()

[{'id': 'hut_wood', 'type': 'objects', 'amount': 1}, {'id': 'resource_workforce', 'type': 'objects', 'amount': 5}, {'id': 'resource_wood', 'type': 'objects', 'amount': 5}]


In [111]:
que.summary

AttributeError: 'Queue' object has no attribute 'summary'

In [167]:
Compiler("scenario", "db.json").compile_scenario()

IsADirectoryError: [Errno 21] Is a directory: 'scenario'

In [36]:
tt = Query()
x.tables["reality"].insert({"name": "Mikolaj"})

1

In [5]:
x.tables["objects"].search(tt.id == "hut_wood")

NameError: name 'x' is not defined

In [39]:
x.tables["reality"].remove(tt.name == "Mikolaj")

[1]

In [40]:
x.tables["reality"].search(tt.name == "Mikolaj")

[]

In [42]:
x.tables.keys()

dict_keys(['reality', 'concepts', 'rules'])

In [62]:
[1,2, 3] == [1,2]

False

In [1]:
%pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 25.9 MB/s eta 0:00:00m eta 0:00:010:01:01
Note: you may need to restart the kernel to use updated packages.
